In [1]:
import requests
import tarfile
import io
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

In [2]:
DATASET_URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

In [3]:
def download_and_extract_data():
    response = requests.get(DATASET_URL)
    tar = tarfile.open(fileobj=io.BytesIO(response.content), mode='r:gz')
    tar.extractall()

In [4]:
def read_imdb_data(path='aclImdb'):
    data = {'review': [], 'sentiment': []}

    for sentiment in ['pos', 'neg']:
        score = 1 if sentiment == 'pos' else 0
        subdir = os.path.join(path, 'train', sentiment)
        for filename in os.listdir(subdir):
            with open(os.path.join(subdir, filename), 'r', encoding='utf-8') as f:
                data['review'].append(f.read())
                data['sentiment'].append(score)

    return pd.DataFrame(data)

In [5]:
def main():
    download_and_extract_data()
    df = read_imdb_data()

    X = df['review']
    y = df['sentiment']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    classifier = LinearSVC()
    classifier.fit(X_train_vec, y_train)

    y_pred = classifier.predict(X_test_vec)

    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))

In [6]:
if __name__ == '__main__':
    main()

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.86      0.87      2485
           1       0.87      0.88      0.87      2515

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000

Accuracy: 0.8706
